## Establish Dependencies and import files

In [1]:
import csv, requests, json, time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
file_loc = 'C:/Users/leala/Documents/Project 1/breweries-brew-pubs-in-the-usa'

file_1 = pd.read_csv(file_loc + '/7160_1.csv')
file_1.head(3)

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites
0,407 Radam Ln,brewery,Austin,US,us/tx/austin/407radamln,NaN,NaN,(512) Brewing Co,5127072337,78745,TX,NaN
1,1135 N W Galveston Ave,Brewery and Bar,Bend,US,us/or/bend/1135nwgalvestonave,NaN,NaN,"10 Barrel Brewing Company, 10 Barrel Brewing Co","(541) 585-1007, 5415851007",97703,OR,10barrel.com
2,830 W Bannock St,"brewery, Restaurant, Pub, Brewery, pub, Brewer...",Boise,US,us/id/boise/830wbannockst,43.617711,-116.202883,"10 Barrel Brewing, 10 Barrel Brewing Co.","(208) 344-5870, 2083445870, 2.08344587E9",83702,ID,10barrel.com


In [3]:
# Is everything in the US?  It should be  but double check
file_1[file_1['country'] != 'US']

,address,categories,city,country,key,lat,long,name,phones,postalCode,province,websites


## Create Dataframe for information within CSV file
- Create list just for TX since that is the focus State

In [4]:
# Create a DataFrame to hold the relavent information
brew_frame = pd.DataFrame({'Name': file_1['name'],
              'Address': file_1['address'],
              'City': file_1['city'],
              'State': file_1['province'],
              'Country': file_1['country'],
              'Zip Code': file_1['postalCode'],
              'Latitude': file_1['lat'],
              'Longitude': file_1['long'],
              'Count': int(1)
              })
                               
# Just show TX because thats the state we're focusing on
brew_frame[brew_frame['State'] == 'TX'].head(3)

,Address,City,Count,Country,Latitude,Longitude,Name,State,Zip Code
0,407 Radam Ln,Austin,1,US,NaN,NaN,(512) Brewing Co,TX,78745
3,1022 Texan Trl,Grapevine,1,US,32.938385,-97.064340,10 Gallon Tours,TX,76051
48,2514 S W 34th Ave,Amarillo,1,US,35.178496,-101.864166,34th Street Discount,TX,79109


## Clean the data within the Dataframe

In [5]:
# look for null information that will throw off full address creation
# brew_nan = brew[brew.isnull().any(axis=1)]
# print(brew_nan)

In [6]:
# Look for dirty data within the State column
brew_frame['State'].unique()

array(['TX', 'OR', 'ID', 'MN', 'CA', 'CO', 'WA', 'MI', 'VT', 'NJ', 'DE',
       'NY', 'IN', 'AZ', 'PA', 'WV', 'ME', 'VA', 'KS', 'MO', 'WI', 'MT',
       'FL', 'DC', 'IL', 'KY', 'OK', 'TN', 'AK', 'GA', 'IA', 'AR', 'NH',
       'OH', 'CT', 'MA', 'NC', 'LA', 'AL', 'WY', 'SC', 'NV', 'HI', 'NM',
       'AU-WA', 'NE', 'MD', 'UT', 'ND', 'RI', 'SD', 'Denver', 'AB', 'NL',
       'MS', 'Los Angeles', 'San Diego', 'Pittsburgh', 'St. Louis',
       'Sacramento', 'ON', 'Tampa'], dtype=object)

In [ ]:
# Replace all the cities in the 'States' column with correct data
brew_frame['State'] = brew_frame.State.str.replace('AU-WA','WA').replace('Denver','CO').replace('Los Angeles','CA')\
                        .replace('San Diego','CA').replace('Sacramento', 'CA').replace('Pittsburgh', 'PA')\
                        .replace('St. Louis', 'MO').replace('Tampa', 'FL')
# double check to ensure States are now uniform state codes
# brew_frame.loc[brew_frame['State'].str.len() > 2]
    
# These show up later as rotten data, fixing them here
brew_frame['City'] = brew_frame.City.str.replace('Saltlakecity ', 'Salt Lake City').replace('Lakeleelanau ', 'Leelanau').replace('Batonrouge', 'Baton Rouge')
brew_frame['Address'] = brew_frame.Address.replace('725 4TH E Saint (btwn D & St)', '725 4TH E Saint').replace('110 N W Shenandoah Ave Hotel Roanoke & Conference Ctr', '110 N W Shenandoah Ave')\
                        .replace('216 7TH Ave (btw 22nd & 23rd)', '216 7TH Ave').replace('30770 E Russell Ranch Road Suites & F', '30770 E Russell Ranch Road')\
                        .replace('317 Broadway (btw 3rd & 4th Ave S)', '317 Broadway').replace('4133 N E University Way (btwn 42nd & 41st)', '4133 N E University Way')\
                        .replace('4517 N Oakland Ave (btwn Kensington & Glendale)', '4517 N Oakland Ave')

# zipcodes are pretty bogus, i'll grab them from the API call instead
# brew_frame.loc[brew_frame['Zip Code'].str.len() > 5].head()

## Graph total Craft Breweries by State

In [8]:
# create count column for dataframe referencing total breweries
state_count = brew_frame.groupby('State')

state_sum = pd.DataFrame({'State': state_count['State'],
                          'State Total': state_count['Count'].sum()
                           })
state_sum.head()


,State,State Total
State,,
AB,"(AB, [AB])",1
AK,"(AK, [AK, AK, AK, AK, AK, AK, AK, AK, AK, AK, ...",27
AL,"(AL, [AL, AL, AL, AL, AL, AL, AL, AL, AL, AL, ...",67
AR,"(AR, [AR, AR, AR, AR, AR, AR, AR, AR, AR, AR, ...",56
AZ,"(AZ, [AZ, AZ, AZ, AZ, AZ, AZ, AZ, AZ, AZ, AZ, ...",155


In [9]:
state_total = state_count.largest(10, columns=['Count'])
state_total

# plt.bar(state_count['State'].head(10), state_count['State Total'].head(10), color='blue', alpha=0.5, align="center")
# plt.title("States with highest number of Breweries")
# plt.xlabel("State Abbreviation")
# plt.ylabel("Total Number of Brewery")

AttributeError: 'DataFrameGroupBy' object has no attribute 'largest'

## API calls to fill in missing Lat and Long information
- Added a more information to the call just to get a complete picture.  May or may not be necessary later

In [10]:
# google places API didnt want to cooperate, using mapquest instead to return lat and long
mq_key = 'nUGqYA2YWuf23HVKSDUqGv4VtOLwvA3F'
address = '407 Radam Ln, Austin TX, 78745'

# create sample API call to get proper index
url = 'http://www.mapquestapi.com/geocoding/v1/address?key='
test = requests.get(url + mq_key + '&location=' + address + '&thumbMaps=false').json()
# print(json.dumps(test, indent=2, sort_keys=True))

# test to make sure calls are pulling correct info
lat_collect = test['results'][0]['locations'][0]['latLng']['lat']
lng_collect = test['results'][0]['locations'][0]['latLng']['lng']

print(test['results'][0]['locations'][0]['postalCode'])

78745-1113


In [12]:
# brew_null = brew_two[brew_two.isnull().any(axis=1)]
# len(brew_null)

In [13]:
full_address= (brew_frame['Address'] + ", " + brew_frame['City'] + " " + brew_frame['State']) 

lat = []
lng = []
zipCode = []
county = []

for address in full_address:
    response = requests.get(url + mq_key + '&location=' + address + '&thumbMaps=false').json()
    time.sleep(1)
    try:
#         print('---------------------------------')
#         print('Location: ', address)
#         print(response['results'][0]['locations'][0]['latLng']['lat'])
#         print(response['results'][0]['locations'][0]['latLng']['lng'])
        lat.append(response['results'][0]['locations'][0]['latLng']['lat'])
        lng.append(response['results'][0]['locatio
                                          ns'][0]['latLng']['lng'])
        zipCode.append(response['results'][0]['locations'][0]['postalCode'])
        county.append(response['results'][0]['locations'][0]['adminArea4'])
    except:
        print(f"Information missing for ", address)
        print(f"-------------------------------------")
        lat.append('NaN')
        lng.append('NaN')
                                          
        zipCode.append('NaN')
        county.append('NaN')
        pass

Information missing for  Corner Of Dixie Hwy & Birch Run Rd, Birch Run MI
-------------------------------------
Information missing for  2290 S French Rd, Lakeleelanau MI
-------------------------------------
Information missing for  Btwn Gates 7 & 8, Madison WI
-------------------------------------
Information missing for  147 W 300 S, Saltlakecity UT
-------------------------------------
Information missing for  2110 S Highland Dr, Saltlakecity UT
-------------------------------------


In [54]:
brew_two = pd.DataFrame({'Brewery': brew_frame['Name'],
                         'Full Address': full_address,
                         'State': brew_frame['State'],
                         'Lat': lat,
                         'Lng': lng,
                         'Zip Code': zipCode,
                         'County': county
                        })
brew_two.head(3)
brew_two.to_csv('breweries.csv')

In [35]:
# get rid of the NaN values I added so they dont get into the plot
# brew_two['Lat'].replace('', np.nan, inplace=True) << from when I didnt just add them in as NaN
# brew_two.dropna(subset=['Lat'], how='any', inplace=True)
print(len(brew_two['Full Address']), len(brew_two['Lat']))

7375 7375


In [40]:
# loop through API requests and parse necessary information

# for index, location in brew_two.iterrows():
#     response = requests.get(url + mq_key + '&inFormat=kvp&outFormat=json&location=' + location[0]).json()
#     try:
#         brew_two.loc[index, 'Lat'] = response['results'][0]['locations'][0]['latLng']['lat']
#         brew_two.loc[index, 'Lng'] = response['results'][0]['locations'][0]['latLng']['lng']
#         brew_two.loc[index, 'Zip Code'] = response['results'][0]['locations'][0]['postalCode']
#         brew_two.loc[index, 'County'] = response['results'][0]['locations'][0]['adminArea4']
#     except IndexError:
#         print(f"Information missing for ", location[0])
#         print(f"-------------------------------------")
#         pass

Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
-------------------------------------
Information missing for  
--------------

KeyboardInterrupt: 

In [36]:
# missing = brew_two.loc[brew_two['Lat'].str.len() < 2]
# print(missing)

## Add everything to a plot to visualize Brewery locations in USA

In [37]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='lamatinal', api_key='VPcrievjz9SJhSSmE5t4')
mapbox_access_token = 'pk.eyJ1IjoibGFtYXRpbmFsIiwiYSI6ImNqa3BxMHk2aDFnbjEzcHA4M3Nhb2Z5N2cifQ.04a2NTZHdUn_KjhPo59zfw'

In [53]:
data = [go.Scattermapbox(lat=brew_two['Lat'], 
                         lon=brew_two['Lng'], 
                         mode='markers',
                         text=brew_two['Brewery'],
                         marker=dict(
                            size=5,
                            color='rgb(0, 0, 128)',
                            opacity=0.5)
                        )
       ]

layout = go.Layout(title='Craft Brewery Locations in the USA', 
                   autosize=True, 
                   hovermode='closest', 
                   showlegend=False,
                   mapbox=dict(accesstoken=mapbox_access_token, bearing=0, center=dict(lat=38, lon=-94),
                    pitch=0,
                    zoom=3,
                    style='light')
                  )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Brewery Locations USA')

In [ ]:
texas = brew_two[brew_two['State'] == 'Texas']

data = [go.Scattermapbox(lat=texas['Lat'], 
                         lon=texas['Lng'], 
                         mode='markers',
                         text=False,
                         marker=dict(size=5)
                        )
       ]

layout = go.Layout(title='Craft Brewery Locations in the USA', 
                   autosize=True, 
                   hovermode='closest', 
                   showlegend=False,
                   mapbox=dict(accesstoken=mapbox_access_token, bearing=0, center=dict(lat=38, lon=-94),
                    pitch=0,
                    zoom=3,
                    style='light')
                  )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Brewery Locations USA')